In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

In [ ]:
import inspect

import numpy as np
import xarray as xr
import xarray_sentinel

from sarsen import apps, geocoding, orbit, scene

# uncomment to check that the code below is in sync with the implementation
# print(inspect.getsource(apps.backward_geocode_sentinel1_slc_iw_burst))

In [ ]:
product_path = (
    "data/S1B_S6_SLC__1SDV_20211216T115438_20211216T115501_030050_03968A_4DCB.SAFE"
)
dem_urlpath = "data/Chicago-4m-DEM.tif"
output_urlpath = "Chicago-4m-SLC.tif"
swath_group = "S6"
swath_polarisation_group = "S6/VV"
measurement_group = "S6/VV"
measurement_block_slices = (slice(12500, 16000), slice(None, 4096))
digital_number_max = 75

!ls -d {product_path}

In [ ]:
measurement = xr.open_dataset(
    product_path, engine="sentinel-1", group=measurement_group
)
measurement

In [ ]:
measurement_block = measurement.measurement[measurement_block_slices]
measurement_block

In [ ]:
_ = abs(measurement_block).plot(y="azimuth_time", vmax=digital_number_max)

In [ ]:
orbit_ecef = xr.open_dataset(
    product_path, engine="sentinel-1", group=f"{swath_polarisation_group}/orbit"
)
orbit_ecef

In [ ]:
dem_raster = scene.open_dem_raster(dem_urlpath)
print(dem_raster.rio.estimate_utm_crs())
dem_raster

In [ ]:
_ = dem_raster.plot(figsize=(12, 8))

In [ ]:
try:
    dem_ecef = scene.convert_to_dem_ecef(dem_raster)
except:
    # first time we run it we get a GDAL exception :shrug:
    dem_ecef = scene.convert_to_dem_ecef(dem_raster)
dem_ecef

In [ ]:
orbit_interpolator = orbit.OrbitPolyfitIterpolator.from_position(orbit_ecef.position)
orbit_interpolator.interval

In [ ]:
azimuth_time = orbit_interpolator.azimuth_time_range()
azimuth_time

In [ ]:
position_sar = orbit_interpolator.position(azimuth_time)
velocity_sar = orbit_interpolator.velocity(azimuth_time)

direction_sar = velocity_sar / np.linalg.norm(velocity_sar)

position_sar

In [ ]:
dem_coords = geocoding.backward_geocode(dem_ecef, position_sar, direction_sar)
dem_coords

In [ ]:
geocoded = abs(measurement.measurement).interp(
    azimuth_time=dem_coords.azimuth_time,
    slant_range_time=dem_coords.slant_range_time,
    method="nearest",
)
geocoded.rio.set_crs(dem_raster.rio.crs)
geocoded

In [ ]:
_ = geocoded.plot(vmax=digital_number_max, figsize=(12, 8))

In [ ]:
geocoded.rio.to_raster(
    output_urlpath,
    tiled=True,
    blockxsize=512,
    blockysize=512,
    compress="DEFLATE",
    num_threads="ALL_CPUS",
)